# OPTIMIZING MODEL PARAMETERS

## Prerequisite Code

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

training_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



## Hyperparameters

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

### Loss Function

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Full Implementation

In [7]:
def train_loop(dataloder, model, loss_fn, optimizer):
    size = len(dataloder.dataset)
    for batch, (X, y) in enumerate(dataloder):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloder, model, loss_fn):
    size = len(dataloder.dataset)
    num_batches = len(dataloder)
    test_loss, corrent = 0, 0
    
    with torch.no_grad():
        for X, y in dataloder:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            corrent += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    corrent /= size
    print(f"Test Error: \n Accuracy:{(100*corrent):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(training_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 2.303346 [    0/60000]
loss: 2.294512 [ 6400/60000]
loss: 2.280084 [12800/60000]
loss: 2.272315 [19200/60000]
loss: 2.249029 [25600/60000]
loss: 2.230813 [32000/60000]
loss: 2.229488 [38400/60000]
loss: 2.194345 [44800/60000]
loss: 2.198025 [51200/60000]
loss: 2.172998 [57600/60000]
Test Error: 
 Accuracy:48.2%, Avg loss: 2.156999 

Epoch 2
-------------------------------
loss: 2.164546 [    0/60000]
loss: 2.155215 [ 6400/60000]
loss: 2.096260 [12800/60000]
loss: 2.114290 [19200/60000]
loss: 2.060450 [25600/60000]
loss: 2.009393 [32000/60000]
loss: 2.028048 [38400/60000]
loss: 1.945755 [44800/60000]
loss: 1.957863 [51200/60000]
loss: 1.893234 [57600/60000]
Test Error: 
 Accuracy:59.0%, Avg loss: 1.877912 

Epoch 3
-------------------------------
loss: 1.907813 [    0/60000]
loss: 1.880034 [ 6400/60000]
loss: 1.754151 [12800/60000]
loss: 1.804465 [19200/60000]
loss: 1.705439 [25600/60000]
loss: 1.652955 [32000/60000]
loss: 1.672834 [38400/60